# Dataset Cleaning
*(This notebook was inspired by Anton T. Ruberts' Dataset Cleaning notebook.)*
This dataset is my own collected data from surveys.

Since the table width is not too large, I decided to clean up the columns via Excel. 

The main objectives of this notebook are:
- Observe the contents of the dataset,
- melt the columns for characters and movies,
- separate data with proper separator from those with improper formatting,
- remove empty entries,
- export the cleaned data.

In [1]:
import pandas as pd
import numpy as np

## Loading the dataset

In [2]:
data = pd.read_csv('../data/Data - movies_MBTI_raw.csv')
data.head(-10)

,Timestamp,"Can you give me something like a username, which only contains alphanumeric characters? Make it random too, to avoid personal information.","Unambiguous Consent\n\nBy clicking ""I agree"" below, you consent to participate in our survey and acknowledge that you have read and understood the data privacy notice provided above. You understand that your participation is voluntary, and you may withdraw your consent and request deletion of your data at any time by contacting us. Additionally, you consent to the publication of general results online.",Age Confirmation\nPlease confirm that you are at least 18 years of age:,What MBTI type did you get the last time you took the test?,What test did you use to get the result above?,Character 1//Movie,Character 2//Movie,Character 3//Movie,Character 4//Movie,Character 5//Movie (optional),Character 6//Movie (optional),Character 7//Movie (optional),Character 8//Movie (optional),Character 9//Movie (optional),Character 10//Movie (optional),"Before leaving, do you have comments, suggestions, or questions?"
0,4/29/2024 15:45:45,deadpool123,I agree.,"Yes, I am at least 18 years old.",ENFP,michaelcaloz.com,Tony Stark//Avengers: Endgame,Captain America//Avengers: Endgame,Miles Morales//Spider-Man: Across the Spider-V...,Hobie Brown//Spider-Man: Across the Spider-Verse,Peter Parker//Spider-Man: Far From Home,Bella Baxter//Poor Things,Jesse Wallace//Before Sunrise,Celine//Before Sunrise,Peter Parker//The Amazing Spider-Man,Rocket Raccoon//Guardians of the Galaxy Vol. 3,NaN
1,4/29/2024 19:59:03,GBDymrBKBcsRMqef9Nyx#Kn#3LiihcF7#&ghPx!M,I agree.,"Yes, I am at least 18 years old.",INTP,16Personalities,Puss in Boots // Puss in Boots: The Last Wish,J. Robert Oppenheimer // Oppenheimer,Johnny English // Johnny English,Parzival (Wade Watts) // Ready Player One,Benoit Blanc // A Knives Out,Dr. Stephen Strange // Doctor Strange,Neo // The Matrix,"Captain Pete ""Maverick"" Mitchell // Top Gun: M...",Tony Stark // Avengers: Infinity War,Loki // Avengers: Infinity War,NaN
2,4/30/2024 4:57:43,Lady-Orpheus,I agree.,"Yes, I am at least 18 years old.",INFP,Truity,Waymond Wang//Everything Everywhere All at Once,Ellen Ripley//Alien,Wall-E//Wall-E,Mathilda//Mathilda,V//V for Vendetta,Morticia//The Addams Family,Amelie Poulain//Amelie,Willy Wonka//Charlie and the Chocolate Factory...,Remus Lupin//Harry Potter,NaN,I'd love to know more about how this data is g...
3,4/30/2024 5:41:28,gyattman123,I agree.,"Yes, I am at least 18 years old.",ENTP,all of the above,Blitzo from Helluva Boss,Jay Bilzerian from Big mouth,Howard from TBBT,Saul Goodman from Better Call Saul,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4/30/2024 8:07:45,Hiii,I agree.,"Yes, I am at least 18 years old.",ENFJ,Sakinorva,Grace Le domas(Ready or Not),Jack(Titanic),Wanda (Marvel),Joy (Inside out),NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,5/4/2024 13:26:07,SDRS13,I agree.,"Yes, I am at least 18 years old.",INFJ,16Personalities,Richie Tozier/IT (2017),"Susan Pevensie/Chronicles of Narnia: The Lion,...",Legolas/Lord of the Rings,Sirius Black/Harry Potter and Order of the Pho...,"Murphy ""Murph"" Cooper/Interstellar",Patrick Stewart/The Perks of Being a Wallflower,Albert Narracott/War Horse,Jasmine/Alladin (Disney),Peter B. Parker/Spiderman: Into the Spiderverse,Hector Rivera/Coco,Based on a charting of favorite characters fro...
109,5/4/2024 15:17:51,97hbjcdtrd56,I agree.,"Yes, I am at least 18 years old.",INFJ,academic setting - MBTI test,Yoda/Empire Strikes Back,Rick Blaine/Casablanca,Theoden/The Two Towers,Jeriba/Enemy Mine,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,5/4/2024 17:16:46,philio756,I agree.,"Yes, I am at least 18 years old.",INFJ,16Personalities,Hermoine Harry Potter,Darcy Pride and prejudice,Anastasia - Anastasia,Will traynor - Me before you,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,5/4/2024 18:06:23,v453v3,I agree.,"Yes, I am at least 18 years old.",INFP,16Personalities,Chloe Beale//Pitch Perfect,The Female/

## Initial exploration of the data

In [3]:
print("Dataset shape:", data.shape)
print("Dataset columns:", data.columns)

Dataset shape: (123, 17)
Dataset columns: Index(['Timestamp',
       'Can you give me something like a username, which only contains alphanumeric characters? Make it random too, to avoid personal information.',
       'Unambiguous Consent\n\nBy clicking "I agree" below, you consent to participate in our survey and acknowledge that you have read and understood the data privacy notice provided above. You understand that your participation is voluntary, and you may withdraw your consent and request deletion of your data at any time by contacting us. Additionally, you consent to the publication of general results online.',
       'Age Confirmation\nPlease confirm that you are at least 18 years of age:',
       'What MBTI type did you get the last time you took the test?',
       'What test did you use to get the result above?', 'Character 1//Movie',
       'Character 2//Movie', 'Character 3//Movie', 'Character 4//Movie',
       'Character 5//Movie (optional)', 'Character 6//Movie (optional

**Observations**
- The columns names are too long.
- Some columns are not necessary.
- Character-movie columns can be simplified. 

**Actions(s)**
- Eliminate "consent" and text-id columns. 
- Insert index column as id.
- Rename character-movie columns as `char_movie1`.

In [5]:
data.drop(['Timestamp', 'Permissions'], axis=1, inplace=True)

### Missing Data

In [6]:
data.isna().sum()

Age                               1
Primary streaming service         1
Hours per day                     0
While working                     3
Instrumentalist                   4
Composer                          1
Fav genre                         0
Exploratory                       0
Foreign languages                 4
BPM                             107
Frequency [Classical]             0
Frequency [Country]               0
Frequency [EDM]                   0
Frequency [Folk]                  0
Frequency [Gospel]                0
Frequency [Hip hop]               0
Frequency [Jazz]                  0
Frequency [K pop]                 0
Frequency [Latin]                 0
Frequency [Lofi]                  0
Frequency [Metal]                 0
Frequency [Pop]                   0
Frequency [R&B]                   0
Frequency [Rap]                   0
Frequency [Rock]                  0
Frequency [Video game music]      0
Anxiety                           0
Depression                  

**Observations**
- There are different scattered sources of the null values (e.g. age, streaming service, listening while working, etc.), and the majority of the missing values come from the BPM, which was mentioned in the forms to leave blank if the participants weren't sure. 

**Outcome(s)**
- We'll delete the former, and retain the latter, as we'll try to impute it through observations.

In [7]:
# Drop all rows with missing values except BPM
data_dropped = data.dropna(subset=data.columns.difference(['BPM']))

# Fill BPM missing with median of BPM
data_filled = data_dropped.fillna(data_dropped['BPM'].median())

# Verify if missing values are filled in
# data_filled.isna().sum()

In [8]:
from scipy.stats import iqr as calc_iqr
def get_outlier_fence(data_column):
    """Returns outlier threshold values

    Args:
        data_column: dataframe column to be observed

    Returns:
        low_threshold, up_threshold: lower and upper threshold values
    """
    iqr = calc_iqr(data_column)
    low_threshold = np.quantile(data_column, 0.25) - 3 * iqr
    up_threshold = np.quantile(data_column, 0.75) + 3 * iqr
    return low_threshold, up_threshold


In [9]:
age_low_th, age_high_th = get_outlier_fence(data_column=data_filled['Age'])
hrs_low_th, hrs_high_th = get_outlier_fence(data_column=data_filled['Hours per day'])
bpm_low_th, bpm_high_th = get_outlier_fence(data_column=data_filled['BPM'])

df_no_outlier = data_filled[
    (data_filled['Age'] >= age_low_th) & (data_filled['Age'] <= age_high_th) &
    (data_filled['Hours per day'] >= hrs_low_th) & (data_filled['Hours per day'] <= hrs_high_th) &
    (data_filled['BPM'] >= bpm_low_th) & (data_filled['BPM'] <= bpm_high_th)
]

In [10]:
df_no_outlier.describe()

,Age,Hours per day,BPM,Anxiety,Depression,Insomnia,OCD
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000
mean,23.599707,3.418328,123.079179,5.906891,4.866569,3.716276,2.680352
std,8.818426,2.498598,31.112220,2.763656,3.004245,3.067127,2.862184
min,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,2.000000,105.000000,4.000000,2.000000,1.000000,0.000000
50%,21.000000,3.000000,120.000000,6.000000,5.000000,3.000000,2.000000
75%,26.000000,5.000000,140.000000,8.000000,7.000000,6.000000,5.000000
max,58.000000,14.000000,220.000000,10.000000,10.000000,10.000000,10.000000


## Data Pre-processing Pipeline

For this coffee dataset, we have a clean dataset such that we do not have to apply a pipeline. Nonetheless, I'm leaving this section here to serve as a template for future projects.

In [11]:
genres_list = list(df_no_outlier.columns)[10:26]
genre_freq_map = {'Never': 0, 'Rarely': 1, 'Sometimes': 2, 'Very frequently': 3}

binary_list = list(['While working', 'Instrumentalist', 'Composer', 'Exploratory', 'Foreign languages'])
binary_map = {'No': 0, 'Yes': 1}

stream_map = {'Other streaming service': 'Others', 'I do not use a streaming service.': 'None'}

numerical_df = df_no_outlier.copy()
numerical_df[genres_list] = df_no_outlier[genres_list].map(lambda x: genre_freq_map.get(x, x))
numerical_df[binary_list] = df_no_outlier[binary_list].map(lambda x: binary_map.get(x, x))
numerical_df['Primary streaming service'] = df_no_outlier['Primary streaming service'].map(lambda x: stream_map.get(x, x))

numerical_df.to_csv('../data/Data - MHxM cleaned.csv', index=False)  

In [12]:
numerical_df

,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,BPM,...,Frequency [Pop],Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,Music effects
2,18.0,Spotify,4.0,0,0,0,Video game music,0,1,132.0,...,1,0,1,1,3,7.0,7.0,10.0,2.0,No effect
4,18.0,Spotify,4.0,1,0,0,R&B,1,0,107.0,...,2,3,3,0,1,7.0,2.0,5.0,9.0,Improve
5,18.0,Spotify,5.0,1,1,1,Jazz,1,1,86.0,...,3,3,3,3,0,8.0,8.0,7.0,7.0,Improve
6,18.0,YouTube Music,3.0,1,1,0,Video game music,1,1,66.0,...,1,1,0,0,2,4.0,8.0,6.0,0.0,Improve
7,21.0,Spotify,1.0,1,0,0,K pop,1,1,95.0,...,2,2,1,0,1,5.0,3.0,5.0,3.0,Improve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,17.0,Spotify,2.0,1,1,0,Rock,1,1,120.0,...,3,0,1,3,0,7.0,6.0,0.0,9.0,Improve
732,18.0,Spotify,1.0,1,1,0,Pop,1,1,160.0,...,3,0,0,2,2,3.0,2.0,2.0,5.0,Improve
733,19.0,Others,6.0,1,0,1,Rap,1,0,120.0,...,2,2,2,1,1,2.0,2.0,2.0,2.0,Improve
734,19.0,Spotify,5.0,1,1,0,Classical,0,0,170.0,...,0,0,0,0,2,2.0,3.0,2.0,1.0,Improve
